3 Part 3 of 4 - CNNs (7 Marks)

3.1 CNN Classifier (1 Mark)

In [1]:
# ==== Part 3.1: CNN classifier on LFW (PyTorch, minimal) ====
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 1) Load LFW (same settings as Part 2)
lfw = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw.images              # shape: [N, H, W], values ~ [0,1]
y = lfw.target
target_names = lfw.target_names
N, H, W = X.shape[0], X.shape[1], X.shape[2]

# 2) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# 3) To torch tensors (add channel dim -> [N,1,H,W])
device = "cuda" if torch.cuda.is_available() else "cpu"
X_train_t = torch.tensor(X_train[:, None, :, :], dtype=torch.float32)
X_test_t  = torch.tensor(X_test[:,  None, :, :], dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
y_test_t  = torch.tensor(y_test,  dtype=torch.long)

# 4) Small Dataset wrapper & loader
from torch.utils.data import TensorDataset, DataLoader
train_ds = TensorDataset(X_train_t, y_train_t)
test_ds  = TensorDataset(X_test_t,  y_test_t)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, drop_last=False)
test_loader  = DataLoader(test_ds,  batch_size=128, shuffle=False, drop_last=False)

# 5) Define CNN: two 3x3 conv (32 ch) + pool + FC
class SimpleCNN(nn.Module):
    def __init__(self, h, w, n_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1,  32, kernel_size=3, padding=1)  # -> [B,32,H,W]
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)  # -> [B,32,H,W]
        self.pool = nn.MaxPool2d(2)                                # halve H,W twice
        # compute flattened dim dynamically
        with torch.no_grad():
            x = torch.zeros(1,1,h,w)
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            self.flat_dim = x.numel()
        self.fc1 = nn.Linear(self.flat_dim, 128)
        self.fc2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # [B,32,H/2,W/2]
        x = self.pool(F.relu(self.conv2(x)))   # [B,32,H/4,W/4]
        x = x.view(x.size(0), -1)              # flatten
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.fc2(x)                        # logits
        return x

n_classes = len(target_names)
model = SimpleCNN(H, W, n_classes).to(device)
print(model)

# 6) Optimizer/Loss
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss() 

# 7) Train
def run_epoch(loader, train=True):
    model.train(train)
    total, correct, total_loss = 0, 0, 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = loss_fn(logits, yb)
        if train:
            opt.zero_grad(); loss.backward(); opt.step()
        total_loss += loss.item() * xb.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == yb).sum().item()
        total += xb.size(0)
    return total_loss/total, correct/total

EPOCHS = 12
for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    te_loss, te_acc = run_epoch(test_loader,  train=False)
    print(f"Epoch {epoch:02d} | train acc={tr_acc:.3f} loss={tr_loss:.3f} | "
          f"test acc={te_acc:.3f} loss={te_loss:.3f}")

# 8) Final evaluation & report
model.eval()
with torch.no_grad():
    logits = model(X_test_t.to(device))
pred = logits.argmax(dim=1).cpu().numpy()
print("Test accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred, target_names=target_names))


SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3456, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
)
Epoch 01 | train acc=0.393 loss=1.751 | test acc=0.413 loss=1.678
Epoch 02 | train acc=0.411 loss=1.696 | test acc=0.413 loss=1.678
Epoch 03 | train acc=0.411 loss=1.718 | test acc=0.413 loss=1.705
Epoch 04 | train acc=0.411 loss=1.667 | test acc=0.413 loss=1.660
Epoch 05 | train acc=0.415 loss=1.614 | test acc=0.450 loss=1.545
Epoch 06 | train acc=0.459 loss=1.503 | test acc=0.512 loss=1.432
Epoch 07 | train acc=0.472 loss=1.428 | test acc=0.506 loss=1.316
Epoch 08 | train acc=0.510 loss=1.284 | test acc=0.618 loss=1.151
Epoch 09 | train acc=0.635 loss=1.103 | test acc=0.643 loss=0.986
Epoch 10 | train acc=0.652 loss=